# Lesson 05D - Multiple Kernel Size with RandomizedSearch


Most of the code here is based on that [post](file:///C:/Users/ishay/Documents/Data%20Science/Coursera%20and%20practice/ML_Mastery/NLP/Model%20Improvement%20and%20Best%20Practices/How%20to%20Develop%20an%20N-gram%20Multichannel%20Convolutional%20Neural%20Network%20for%20Sentiment%20Analysis.html) from ML Mastery.

In [27]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
#from keras.models import Sequential
from keras.models import Model
from keras.layers import Embedding
from keras.layers import SpatialDropout1D
from keras.layers import Convolution1D
from keras.layers import Conv1D
from keras.layers import GlobalMaxPooling1D
from keras.layers import MaxPooling1D
from keras.layers import Input
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.merge import concatenate

In [28]:
max_features = 5000

In [29]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features) # instead on nb_words=

In [30]:
X_train.shape

(25000,)

**Sequence Padding**

In [31]:
maxlen = 400

We define maximum length of 400. It means that posts shoeter than 400 will be filled with 0s, and longer posts are cut.

In [32]:
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

**Now defining the model**

**!!Keras wrapper don't support multi input network. So if I want to use gridsearch or randomized search with multi-channel model, I must use one input only!!**

In [33]:
def create_model(kernel_size_conv1=3, n_strides=1, output_vec_dim=50):
    inputs_shape = Input(shape=(400,))
    # First channel
    #inputs_shape1 = Input(shape=(400,))
    embed1 = Embedding(input_dim=5000, output_dim=output_vec_dim)(inputs_shape)
    spat_drop1 = SpatialDropout1D(0.4)(embed1)
    conv1 = Conv1D(filters=250, kernel_size=kernel_size_conv1, padding='valid', activation='relu', strides=n_strides)(spat_drop1)
    pool1 = MaxPooling1D(pool_size=2)(conv1)
    flat1 = Flatten()(pool1)
    
    # Second channel
    #inputs_shape2 = Input(shape=(400,))
    embed2 = Embedding(input_dim=5000, output_dim=output_vec_dim)(inputs_shape)
    spat_drop2 = SpatialDropout1D(0.3)(embed2)
    conv2 = Conv1D(filters=250, kernel_size=4, padding='valid', activation='relu', strides=n_strides)(spat_drop2)
    pool2 = MaxPooling1D(pool_size=2)(conv2)
    flat2 = Flatten()(pool2)
    
    # Third channel
    #inputs_shape3 = Input(shape=(400,))
    embed3 = Embedding(input_dim=5000, output_dim=output_vec_dim)(inputs_shape)
    spat_drop3 = SpatialDropout1D(0.4)(embed3)
    conv3 = Conv1D(filters=250, kernel_size=5, padding='valid', activation='relu', strides=n_strides)(spat_drop3)
    pool3 = MaxPooling1D(pool_size=2)(conv3)
    flat3 = Flatten()(pool3)
    # merge
    merged = concatenate([flat1, flat2, flat3])
    # FC layers
    dense1 = Dense(250, activation='relu')(merged)
    drop1 = Dropout(0.3)(dense1)
    dense2 = Dense(20, activation='relu')(drop1)
    drop2 = Dropout(0.3)(dense2)
    output = Dense(1, activation='sigmoid')(drop2)
    model = Model(inputs=inputs_shape, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [34]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

In [35]:
model = KerasClassifier(build_fn=create_model, verbose=1, epochs=3, batch_size=32)

In [36]:
create_model().summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_16 (Embedding)        (None, 400, 50)      250000      input_16[0][0]                   
__________________________________________________________________________________________________
embedding_17 (Embedding)        (None, 400, 50)      250000      input_16[0][0]                   
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, 400, 50)      250000      input_16[0][0]                   
__________________________________________________________________________________________________
spatial_dr

In [37]:
param_dist= {"kernel_size_conv1": [2,3],
             "n_strides": sp_randint(1,3),
             "output_vec_dim": sp_randint(60,100)}

In [38]:
random_grid = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter = 3, n_jobs=1, return_train_score=True, cv=2)
random_grid_result = random_grid.fit(X_train, y_train)

Epoch 1/3
 3264/12500 [======>.......................] - ETA: 3:18 - loss: 0.7492 - acc: 0.4890

KeyboardInterrupt: 

In [26]:
X_train.shape

(25000, 400)

In [ ]:
data.reshape((999,1))

**Fitting the model**

I must enter X_train three times (also X_test), one for each channel

In [64]:
epochs = 5
batch_size = 32
h = model.fit([X_train, X_train, X_train], y_train, batch_size=batch_size, epochs=epochs, 
              validation_data=([X_test,X_test,X_test], y_test), verbose=1)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
  512/25000 [..............................] - ETA: 7:55 - loss: 0.7903 - acc: 0.5137

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=0)
print("Training: accuracy = %f  ;  loss = %f" % (accuracy, loss))

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Validation: accuracy1 = %f  ;  loss1 = %f" % (accuracy, loss))

In [ ]:
# A good way to view the shape of each layer. Can be printed after the compile function
print(model.summary())